# CHPyT


I'll be sharing here some examples about how to use the code that I am developing, just in case you also want to use it. The code is in progress so I will be explaining its functionalities when I actually implement them

The code is very basic and I am open to every type of suggestions (about its desing, the algorithms, naming choices, etc...)

### File poly.py

This file contains the basics of symbolic expression manipulation. It allows you to create Symbols and multpliply/add them to create expressions.

Right now it is only capable of showing expanded results, i.e. without any commom factor ((3-a)(b+c) will be shown as 3b+3c-ab-ac). I believe that showing factorized results should not be very hard to do and I may implement it in some near future

As this is a basic code it can only work with simple polynomical expressions. Let's create some symbols!

In [12]:
import poly

In [13]:
a = poly.Symbol("a")
b = poly.Symbol("b")
c = poly.Symbol("c")
d = poly.Symbol("d")

This will allow you to perform expression manipulation as would do in any other software. **Please tell me if you encounter any bugs!**

In [14]:
expr = (2*a-b)*(3*c+5*d**2)
expr

6*a*c + 10*a*d^2 + -3*b*c + -5*b*d^2

### File physics.py

This file contains the objects needed to perform the ChPT computations. The first object that we main want to create is a Scalar Field

In [15]:
import physics
import greeksymbols as gs

In [17]:
x = poly.Symbol("x")

# To create an ScalarField we need to provide something to act as a symbol and the point in which it is evaluated (for now it is not important)
pi0 = physics.ScalarField(gs.pi+"0","x")
pip = physics.ScalarField(gs.pi+"⁺","x")
pim = physics.ScalarField(gs.pi+"⁻","x")

# Something important is that you need to link corresponding antiparticles.
# If a particle is not linked an antiparticle, it will assume it is its own antiparticle.
# This will affect how wick's contractions are taken
physics.ScalarField.link_antiparticles(pip,pim)

Now we can perform operations with these Scalar Fields

In [22]:
expr = 2*pi0**2*pip*pim
expr

2*π0(x)^2*π⁺(x)*π⁻(x)

We can also take derivatives of the fields. We need to specify some index (also a symbol)

In [23]:
mu = poly.Symbol(gs.mu)

partial_expr = pi0.partial(mu)
partial_expr

∂_μ(π0(x))

We can also derivate whole expressions

In [24]:
print("Expression before partial")
print(expr)
print("Expression after partial")
print(expr.partial(mu))

Expression before partial
2*π0(x)^2*π⁺(x)*π⁻(x)
Expression after partial
4*π0(x)*π⁺(x)*π⁻(x)*∂_μ(π0(x)) + 2*π0(x)^2*π⁻(x)*∂_μ(π⁺(x)) + 2*π0(x)^2*π⁺(x)*∂_μ(π⁻(x))


The cool thing is that we can use these custom made objects within numpy's functions. For example, for SU(2) chpt:

In [25]:
import numpy as np

In [27]:
phi = np.array([[(1/np.sqrt(2))*pi0, pip],[pim, -(1/np.sqrt(2))*pi0]])
phi

array([[0.7071067811865475*π0(x), π⁺(x)],
       [π⁻(x), -0.7071067811865475*π0(x)]], dtype=object)

Note that sqrts and fractions are evaluated numerically. I want to change this in the future

Now, we can compute the Lagrangian:

In [30]:
import copy

In [33]:
f = poly.Symbol("f")
U = np.eye(2) + 1j*np.sqrt(2)*f**(-1)*phi - 1*f**(-2)*np.matmul(phi,phi) - 1j*(np.sqrt(2)/3)*f**(-3)*np.matmul(phi,np.matmul(phi,phi))
Udag = np.eye(2) - 1j*np.sqrt(2)*f**(-1)*phi - 1*f**(-2)*np.matmul(phi,phi) + 1j*(np.sqrt(2)/3)*f**(-3)*np.matmul(phi,np.matmul(phi,phi))

# The function partial does not work with numpy arrays (yet) so we need to iterate over the matrix elements.
pU = copy.deepcopy(U)
pUdag = copy.deepcopy(Udag)
for ii in range(len(pU)):
    for jj in range(len(pU)):
        pU[ii,jj] = pU[ii,jj].partial(mu)
        pUdag[ii,jj] = pUdag[ii,jj].partial(mu)

In [35]:
Lag2 = f**2*(1/4)*np.trace(np.matmul(pUdag,pU))

In [36]:
Lag2

(0.5+0j)*∂_μ(π0(x))^2 + (0.24999999999999978+0j)*f^(-2)*π0(x)^2*∂_μ(π0(x))^2 + (-0.16666666666666666+0j)*f^(-2)*π⁺(x)*π⁻(x)*∂_μ(π0(x))^2 + (0.6666666666666665+0j)*f^(-2)*π0(x)*π⁻(x)*∂_μ(π0(x))*∂_μ(π⁺(x)) + (0.6666666666666665+0j)*f^(-2)*π0(x)*π⁺(x)*∂_μ(π0(x))*∂_μ(π⁻(x)) + (0.3333333333333333+0j)*f^(-2)*π⁻(x)^2*∂_μ(π⁺(x))^2 + (0.33333333333333326+0j)*f^(-2)*π⁺(x)*π⁻(x)*∂_μ(π⁺(x))*∂_μ(π⁻(x)) + (0.3333333333333333+0j)*f^(-2)*π⁺(x)^2*∂_μ(π⁻(x))^2 + (-0.25+0j)*f^(-2)*π0(x)^2*∂_μ(π0(x))^2 + (0.125+0j)*f^(-4)*π0(x)^4*∂_μ(π0(x))^2 + (0.27777777777777773+0j)*f^(-4)*π0(x)^2*π⁺(x)*π⁻(x)*∂_μ(π0(x))^2 + (0.2222222222222222+0j)*f^(-4)*π0(x)^3*π⁻(x)*∂_μ(π0(x))*∂_μ(π⁺(x)) + (0.2222222222222222+0j)*f^(-4)*π0(x)^3*π⁺(x)*∂_μ(π0(x))*∂_μ(π⁻(x)) + (-0.16666666666666666+0j)*f^(-2)*π⁺(x)*π⁻(x)*∂_μ(π0(x))^2 + (0.05555555555555555+0j)*f^(-4)*π⁺(x)^2*π⁻(x)^2*∂_μ(π0(x))^2 + (0.4444444444444444+0j)*f^(-4)*π0(x)*π⁺(x)*π⁻(x)^2*∂_μ(π0(x))*∂_μ(π⁺(x)) + (0.4444444444444444+0j)*f^(-4)*π0(x)*π⁺(x)^2*π⁻(x)*∂_μ(π0(x))*∂_μ(

### Diagram generation

If we want to generate a diagram we need to first define the correlation function that we want to study. For example the 4 point pip pip -> pip pip: 

In [40]:
# I will call the external points a, b, c, and d
corr = pip.change_point(a)*pip.change_point(b)*pim.change_point(c)*pim.change_point(d)

corr

π⁺(a)*π⁺(b)*π⁻(c)*π⁻(d)

Now we need to pass a list of the operators that the want to insert, for example if we want to generate two insertions of the operator $\pi^+*\pi^-*\pi0^2$ then:

In [41]:
operator = pip*pim*pi0**2

Now we can generate diagramas as follows:

In [42]:
diagrams = physics.Diagram.generate(corr,[operator,operator])
diagrams

[2]D_π0(int1-int2)^2*D_π⁻(d-int2)*D_π⁺(b-int2)*D_π⁻(c-int1)*D_π⁺(a-int1) + [2]D_π0(int1-int2)^2*D_π⁻(d-int1)*D_π⁺(b-int2)*D_π⁻(c-int2)*D_π⁺(a-int1) + [2]D_π0(int1-int2)^2*D_π⁺(b-int1)*D_π⁻(d-int2)*D_π⁻(c-int1)*D_π⁺(a-int2) + [2]D_π0(int1-int2)^2*D_π⁺(b-int1)*D_π⁻(d-int1)*D_π⁻(c-int2)*D_π⁺(a-int2)

The internal points are called "intii" where ii in an index. This diagrams should have the correct symmetry factor

The idea is to iterate over the lagrangian operators to generate all desired diagrams.
Right now, if an operator has a derivative, it should give the correct sym factor but it will not do the Fourier transform so no momentum will appear. (I will soon implement this but I am thinking about what is the best way of doing this)